# Creating a Service Spike:

- Language recognition

- Audio Emotion Recognition

- Star model (reviews)

- Name Entity Recognition

- Summarization

- Question- Answering

Trying to create a little report about the person describing him/herself and obtain a continuous scale in [1,5] for product reviews.

If we have some long reviews (more than 100 words), we could use summarization and then the Star model.

All the presented models are pre-trained with a high accuracy. Some of them are SOTA models.



### Language Recognition Using SOTA model

[Link](https://huggingface.co/speechbrain/lang-id-commonlanguage_ecapa) do modelo!

In [8]:
#import torchaudio
#from speechbrain.pretrained import EncoderClassifier
#import torch
#import librosa
from transformers import HubertForSequenceClassification, Wav2Vec2FeatureExtractor
from transformers import pipeline

In [ ]:
classifier = EncoderClassifier.from_hparams(source="../speechbrain/lang-id-commonlanguage_ecapa", savedir="pretrained_models/lang-id-commonlanguage_ecapa")
# English
out_prob, score, index, text_lab = classifier.classify_file('../data/Actor_01/03-01-01-01-01-01-01.wav')
print(text_lab)

# Spanish
out_prob, score, index, text_lab = classifier.classify_file('../language_data/english/audio_example.wav')
print(text_lab)

# Spanish
out_prob, score, index, text_lab = classifier.classify_file('../language_data/spanish/audio_0.wav')
print(text_lab)

This could be an input for the speech to text model or the emotion model, if we will have one for each language. Then, we can calculate emotions, [Link](https://huggingface.co/superb/hubert-large-superb-er). My docker Kernel didn't support the memory resources needed to run the model. Using the model online until have required resources (```audio_example.wav```):

### English audio: female speaking about design projects (happy-neutral), 22 seconds, 16KHz
```
[
  {
    "score": 0.6868414282798767,
    "label": "hap"
  },
  {
    "score": 0.20297573506832123,
    "label": "neu"
  },
  {
    "score": 0.08876093477010727,
    "label": "sad"
  },
  {
    "score": 0.021421853452920914,
    "label": "ang"
  }
]
````


The results are similar to those models trained locally: emociones forzadas son correctamente reconocidas, habla neutral con problemas (asociado a calma o tristeza).

## Star scale for English-Spanish reviews. 

Si el test contiene una pregunta particular sobre gusto-opción, existe un modelo capaz de puntuar con estrellas esos comentarios [Link](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment). Por ejemplo, el comentario:

*No me parece una característica importante, realmente no entendí lo del audio. Pero creo que podría mejorar bastante. A pesar de lo anterior me gustaron muchos los colores y la interfaz gráfica, creo que seguiría utilizando su herramienta.*

Se obtiene el siguiente resultado:

```
[
  [
    {
      "label": "1 star",
      "score": 0.024148158729076385
    },
    {
      "label": "2 stars",
      "score": 0.3932456076145172
    },
    {
      "label": "3 stars",
      "score": 0.5413529872894287
    },
    {
      "label": "4 stars",
      "score": 0.03902066498994827
    },
    {
      "label": "5 stars",
      "score": 0.0022325613535940647
    }
  ]
]
```

Otro ejemplo de un review positivo: 

*Me encantó la herramienta, buenísimo. Sobre todo la rapidez. *

```
[
  [
    {
      "label": "1 star",
      "score": 0.0010152067989110947
    },
    {
      "label": "2 stars",
      "score": 0.0008651533862575889
    },
    {
      "label": "3 stars",
      "score": 0.005407069344073534
    },
    {
      "label": "4 stars",
      "score": 0.09633979201316833
    },
    {
      "label": "5 stars",
      "score": 0.8963727951049805
    }
  ]
]
```
Y finalmente un comentario negativo:

*No, no me gustó, no sé , no entiendo bien cómo pedir el crédito. *

```
[
  [
    {
      "label": "1 star",
      "score": 0.5617536902427673
    },
    {
      "label": "2 stars",
      "score": 0.3803412616252899
    },
    {
      "label": "3 stars",
      "score": 0.05539131909608841
    },
    {
      "label": "4 stars",
      "score": 0.0019645506981760263
    },
    {
      "label": "5 stars",
      "score": 0.000549106567632407
    }
  ]
]
```



### NER

El siguiente modelo de Hugging Face [Link](https://huggingface.co/flair/ner-spanish-large) distingue entre persona, locación, organización y miscelaneos:

*Hola mi nombre es Rodolfo Lobo, estoy actualmente en Chile, soy doctor en matemáticas de la Universidad de Campinas y actualmente trabajo en Globant como cientista de datos.* 

Obteniendo como resultado:

Hola mi nombre es Rodolfo Lobo[PER], estoy actualmente en Chile[LOC], soy doctor en matemáticas de la Universidad de Campinas[ORG] y actualmente trabajo en Globant[ORG] como cientista de datos.

JSON output

```
[
  {
    "entity_group": "PER",
    "word": "Rodolfo Lobo",
    "start": 18,
    "end": 30,
    "score": 0.9999709725379944
  },
  {
    "entity_group": "LOC",
    "word": "Chile",
    "start": 53,
    "end": 58,
    "score": 0.9999845027923584
  },
  {
    "entity_group": "LOC",
    "word": "Santiago",
    "start": 63,
    "end": 71,
    "score": 0.9999725818634033
  },
  {
    "entity_group": "LOC",
    "word": "Maipú",
    "start": 82,
    "end": 87,
    "score": 0.999947190284729
  },
  {
    "entity_group": "ORG",
    "word": "Universidad de Campinas",
    "start": 121,
    "end": 144,
    "score": 0.9999004205067953
  },
  {
    "entity_group": "ORG",
    "word": "Globant",
    "start": 170,
    "end": 177,
    "score": 0.9999158382415771
  }
]
```

## Summarization

El modelo utilizado es [Link](https://huggingface.co/mrm8488/bert2bert_shared-spanish-finetuned-summarization)

Utilizando la transcripción de las entrevistas de augmented design, por ejemplo:
    
* Y ahorita lo que me lo único que me imagino es poder ver un agregado de Cuál ha sido la respuesta emocional de del usuario de la donde los usuarios de varios usuarios en resumen cual ha sido el feeling general luego de que me han dado respuesta otra cosa que me gustaría ver es poder leer la respuesta el texto y al mismo tiempo ver cuál cuál ha sido su evolución emocional mientras me habla porque si me va a hablar de una misión entiendo que va a realizar varias acciones* 

Obtengo como resumen desde el modelo: 

* Y ahorita lo que me lo único que me imagino es poder ver un agregado de Cuál ha sido la respuesta emocional de un usuario de la donde los usuarios de varios usuarios en resumen cuál ha sido el feeling general luego de que me han dado respuesta otra cosa *

JSON output
```
[
  {
    "summary_text": "Y ahorita lo que me lo único que me imagino es poder ver un agregado de Cuál ha sido la respuesta emocional de un usuario de la donde los usuarios de varios usuarios en resumen cuál ha sido el feeling general luego de que me han dado respuesta otra cosa"
  }
]
```

Y la valoración de ese texto con el modelo Star fue:

```   
    [
  [
    {
      "label": "1 star",
      "score": 0.1152268648147583
    },
    {
      "label": "2 stars",
      "score": 0.15568290650844574
    },
    {
      "label": "3 stars",
      "score": 0.2357410192489624
    },
    {
      "label": "4 stars",
      "score": 0.28364041447639465
    },
    {
      "label": "5 stars",
      "score": 0.20970876514911652
    }
  ]
]
```

- Todas estas funcionalidades permitirían crear un mini reporte de la persona que realizará el test, sin necesidad de anotación manual, en caso de solicitar una presentación formal sobre quién es y qué hace.

- El reporte contiene: nombre, edad, locación o miscelaneo, idioma.

- En caso de preguntas de valoración tendríamos una medida de los comentarios en una escala de 1 a 5 continua. 

- También podríamos usar un modelo doble o triple para reconocer inflexiones emocionales en la voz (en construcción). 

- El reconocimiento de lenguaje será importante en caso de no contar con un servicio speech to text capaz de facilitar ese paso. 

### Question- Answering

En el futuro teniendo más respuestas de una misma pregunta podemos probar este modelo [Link](https://huggingface.co/eduardofv/stsb-m-mt-es-distiluse-base-multilingual-cased-v1), por ejemplo:

¿Crees tú que sería útil utilizar el audio y las emociones en nuestra herramienta?

Source Sentence: Si sirve la información del audio

```Respuesta_1 :``` si me parece que es bastante útil, pero requiere un trabajo adicional, no, si vas aprovechar esto tienes que saber incorporar esta capa emocional y hacer el trabajo de ver como lo voy a , como voy a trabajar con un plan de de cambios y avances basados en esto también. No es solamente oh el resultado de la evaluación es esto, entonces, creo que hay un trabajo adicional para hacer, para aprovecharlo al máximo. 

```Respuesta_2:``` Podría ser, a mi me serviría quizás mostrarle un poco de estos audios a los stakeholders, porque por ejemplo me pasó en Itaú que yo fuí al call-center, no sé si tu fuiste alguna vez al call-center, pero yo fuí a escuchar la gente como reclamaba, porque dije, quiero oír la experiencia, quiero oir que pasa acá, quiero ver si en verdad los números que dicen que está todo mal

```Respuesta_3 (fuera de contexto):``` Quisiera ver otro color dentro de la pantalla, y también no me gustó como se desplegó el menu. Sin embargo, creo que son cosas fáciles de mejorar y todo el resto me pareció claro.

Resumiendo las preguntas usando el modelo anterior: 

```Respuesta_1 Resumen:``` Si me parece que es bastante útil, pero requiere un trabajo adicional, no, si vas aprovecharlo al máximo
    
    
    
```Respuesta_2 Resumen:```Podría ser, a mi me serviría quizás mostrarle un poco de estos audios a los stakeholders, porque por ejemplo me pasó en Itaú que yo fuí al call - center, no sé si tu fuiste alguna vez.

```Respuesta_3 Resumen (negación de la anterior):```No lo creo, a mi no me serviría mostrarle audios a los stakeholders.

```Respuesta_4 Resumen:```Quisiera ver otro color dentro de la pantalla, y también no me gustó como se desplegó el menu. Sin embargo, creo que son cosas fáciles de mejorar y todo el resto me pareció claro

Respuesta:
```
[
  0.5746381282806396,
  0.14650726318359375,
  0.2676565945148468,
  0.02673211507499218
]
 ```
Es decir, la última frase es la que tiene menos relación con la premisa de la pregunta. Por lo tanto, las tres primeras personas están hablando de lo mismo. Pasando ahora por un modelo de [Sentiment Analysis](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment) podemos averiguar el valor de cada frase dentro del mismo contexto, usando el modelo de estrellas:


En paréntesis está mi valoración de la respuesta, en los JSON la realizada por el modelo de estrellas:

```Respuesta_1``` (neutral+positiva) 

```

[
  [
    {
      "label": "1 star",
      "score": 0.02460716851055622
    },
    {
      "label": "2 stars",
      "score": 0.22736528515815735
    },
    {
      "label": "3 stars",
      "score": 0.6106325387954712
    },
    {
      "label": "4 stars",
      "score": 0.1263936161994934
    },
    {
      "label": "5 stars",
      "score": 0.011001372709870338
    }
  ]
]
```

```Respuesta_2``` (neutral-positiva)

```
[
  [
    {
      "label": "1 star",
      "score": 0.3197609782218933
    },
    {
      "label": "2 stars",
      "score": 0.38994067907333374
    },
    {
      "label": "3 stars",
      "score": 0.22866202890872955
    },
    {
      "label": "4 stars",
      "score": 0.045774683356285095
    },
    {
      "label": "5 stars",
      "score": 0.015861690044403076
    }
  ]
]
```

```Respuesta_3``` (negativa)

```
[
  [
    {
      "label": "1 star",
      "score": 0.3477567732334137
    },
    {
      "label": "2 stars",
      "score": 0.4810763895511627
    },
    {
      "label": "3 stars",
      "score": 0.1606052964925766
    },
    {
      "label": "4 stars",
      "score": 0.008705786429345608
    },
    {
      "label": "5 stars",
      "score": 0.0018557929433882236
    }
  ]
]
```
